# Azure chat completion models with your own data (preview)

> 注意: openaiライブラリの新しいバージョンが利用可能です。https://github.com/openai/openai-python/discussions/742 をご覧ください

この例では、Azure OpenAIサービスモデルを独自のデータと組み合わせて使用する方法を示します。この機能は現在プレビュー段階です。

Azure OpenAI on your dataを使用すると、モデルの訓練や微調整を行うことなく、GPT-3.5-TurboやGPT-4などのサポートされているチャットモデルを独自のデータ上で実行できます。データ上でモデルを実行することで、より高い精度と速度でデータの上でチャットし、分析することが可能になります。Azure OpenAI on your dataの主要な利点の一つは、会話型AIのコンテンツをカスタマイズできることです。モデルが特定のソースにアクセスでき、回答をサポートするために参照できるため、答えは事前訓練された知識だけでなく、指定されたデータソースで利用可能な最新情報にも基づいています。この基盤データは、モデルが古い情報や不正確な情報に基づいた回答を生成することを避けるのにも役立ちます。

Azure Cognitive SearchとAzure OpenAI on your own dataは、知識検索のためのカスタマイズ可能で事前構築されたソリューションを提供し、これを基に会話型AIアプリケーションを構築できます。知識検索とセマンティック検索の代替手法については、[ベクターデータベース](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases)のクックブック例をご確認ください。

## 仕組み

[Azure OpenAI on your own data](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data) は、モデルをあなたのデータと接続し、モデルの出力を向上させる方法でデータを取得・活用する能力を提供します。Azure Cognitive Search と組み合わせることで、ユーザーの入力と提供された会話履歴に基づいて、指定されたデータソースからデータが取得されます。そのデータは拡張され、プロンプトとしてモデルに再送信されることで、モデルが応答を生成する際に使用できる文脈情報を提供します。

詳細については、[Azure OpenAI Service のデータ、プライバシー、セキュリティ](https://learn.microsoft.com/legal/cognitive-services/openai/data-privacy?context=%2Fazure%2Fai-services%2Fopenai%2Fcontext%2Fcontext) をご覧ください。

## 前提条件
開始するにあたり、いくつかの前提条件について説明します。

Azure OpenAI Serviceに適切にアクセスするには、[Azure Portal](https://portal.azure.com)で適切なリソースを作成する必要があります（詳細な手順については[Microsoft Docs](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)のガイドを参照してください）。

Azure OpenAIモデルで独自のデータを使用するには、以下が必要です：

1. Azure OpenAIへのアクセスと、チャットモデルがデプロイされたリソース（例：GPT-3またはGPT-4）
2. Azure Cognitive Searchリソース
3. Azure Blob Storageリソース
4. データとして使用するドキュメント（[データソースオプション](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data#data-source-options)を参照）

ドキュメントをBlob Storageにアップロードし、Azure AI Studioを使用してインデックスを作成する方法の完全なチュートリアルについては、この[クイックスタート](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?pivots=programming-language-studio&tabs=command-line)を参照してください。

## セットアップ

まず、必要な依存関係をインストールします。

In [ ]:
! pip install "openai>=0.28.1,<1.0.0"
! pip install python-dotenv

この例では、`dotenv`を使用して環境変数を読み込みます。Azure OpenAIと検索インデックスに接続するために、以下の変数を`KEY=VALUE`形式で`.env`ファイルに追加する必要があります：

* `OPENAI_API_BASE` - Azure OpenAIエンドポイント。これはAzure PortalのAzure OpenAIリソースの「キーとエンドポイント」で確認できます。
* `OPENAI_API_KEY` - Azure OpenAI APIキー。これはAzure PortalのAzure OpenAIリソースの「キーとエンドポイント」で確認できます。Azure Active Directory認証を使用する場合は省略してください（下記の`Microsoft Active Directoryを使用した認証`を参照）
* `SEARCH_ENDPOINT` - Cognitive Searchエンドポイント。このURLはAzure Portalの検索リソースの「概要」で確認できます。
* `SEARCH_KEY` - Cognitive Search APIキー。Azure Portalの検索リソースの「キー」で確認できます。
* `SEARCH_INDEX_NAME` - 独自のデータで作成したインデックスの名前。

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

In [2]:
openai.api_base = os.environ["OPENAI_API_BASE"]

# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"

### 認証

Azure OpenAI サービスは、APIキーとAzure認証情報を含む複数の認証メカニズムをサポートしています。

In [3]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_type`を`azure`に設定し、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の*「キーとエンドポイント」*で確認できます）。

In [4]:
if not use_azure_active_directory:
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]

#### Microsoft Active Directoryを使用した認証
Microsoft Active Directory認証を使用してキーを取得する方法を見てみましょう。設定方法の詳細については、[ドキュメント](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)を参照してください。

In [ ]:
! pip install azure-identity

In [5]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = "azure_ad"
    openai.api_key = token.token

トークンは一定期間有効で、その後期限切れになります。すべてのリクエストで有効なトークンが送信されるようにするには、`requests.auth`にフックすることで期限切れのトークンを更新できます：

In [8]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req


## 独自データを使用したチャット補完モデル

### コンテキストの設定

この例では、モデルがAzure AIサービスのドキュメントデータに基づいて応答するようにしたいと考えています。前述の[クイックスタート](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line&pivots=programming-language-studio)に従って、[Azure AIサービスと機械学習](https://learn.microsoft.com/azure/ai-services/cognitive-services-and-machine-learning)ドキュメントページの[markdown](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/ai-services/cognitive-services-and-machine-learning.md)ファイルを検索インデックスに追加しました。これで、モデルはAzure AIサービスと機械学習に関する質問に答える準備が整いました。

申し訳ございませんが、翻訳すべきテキストが「### Code」という見出しのみのようです。

「### Code」は日本語では「### コード」と翻訳されます。

もし他に翻訳したい内容がございましたら、完全なテキストを提供していただければと思います。

Azure OpenAI モデルと Python SDK を使用して独自のデータでチャットするには、まず独自のデータと連携するように設計されたチャット補完拡張エンドポイントをターゲットとするようにコードを設定する必要があります。これを行うために、指定されたデプロイメント ID の拡張エンドポイントをターゲットとするライブラリ用のカスタムアダプターを設定するために呼び出すことができる便利な関数を作成しました。

In [5]:
import requests

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
    
    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    if use_azure_active_directory:
        session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session


これで、独自のデータに使用する予定のモデルでSDKを設定するための便利な関数を呼び出すことができます。

In [6]:
setup_byod("gpt-4")

`dataSources`キーワード引数に検索エンドポイント、キー、インデックス名を提供することで、モデルに投げかけられる質問は私たちのデータに基づいて回答されるようになります。追加のプロパティである`context`が提供され、モデルが質問に答えるために参照したデータが表示されます。

In [7]:
completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    deployment_id="gpt-4",
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ["SEARCH_ENDPOINT"],
                "key": os.environ["SEARCH_KEY"],
                "indexName": os.environ["SEARCH_INDEX_NAME"],
            }
        }
    ]
)
print(completion)

{
  "id": "65b485bb-b3c9-48da-8b6f-7d3a219f0b40",
  "model": "gpt-4",
  "created": 1693338769,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Azure AI services and Azure Machine Learning (AML) both aim to apply artificial intelligence (AI) to enhance business operations, but they target different audiences and offer different capabilities [doc1]. \n\nAzure AI services are designed for developers without machine learning experience and provide pre-trained models to solve general problems such as text analysis, image recognition, and natural language processing [doc5]. These services require general knowledge about your data without needing experience with machine learning or data science and provide REST APIs and language-based SDKs [doc2].\n\nOn the other hand, Azure Machine Learning is tailored for data scientists and involves a longer process of data co

モデルからのレスポンスをストリーミングしたい場合は、`stream=True`キーワード引数を渡すことができます：

In [8]:
response = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    deployment_id="gpt-4",
    dataSources=[
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ["SEARCH_ENDPOINT"],
                "key": os.environ["SEARCH_KEY"],
                "indexName": os.environ["SEARCH_INDEX_NAME"],
            }
        }
    ],
    stream=True,
)

for chunk in response:
    delta = chunk.choices[0].delta

    if "role" in delta:
        print("\n"+ delta.role + ": ", end="", flush=True)
    if "content" in delta:
        print(delta.content, end="", flush=True)
    if "context" in delta:
        print(f"Context: {delta.context}", end="", flush=True)

Context: {
  "messages": [
    {
      "role": "tool",
      "content": "{\"citations\":[{\"content\":\"<h2 id=\\\"how-are-azure-ai-services-and-azure-machine-learning-aml-similar\\\">How are Azure AI services and Azure Machine Learning (AML) similar?.</h2>\\n<p>Both have the end-goal of applying artificial intelligence (AI) to enhance business operations, though how each provides this in the respective offerings is different..</p>\\n<p>Generally, the audiences are different:</p>\\n<ul>\\n<li>Azure AI services are for developers without machine-learning experience..</li>\\n<li>Azure Machine Learning is tailored for data scientists.\",\"id\":null,\"title\":\"Azure AI services and machine learning\",\"filepath\":\"cognitive-services-and-machine-learning.md\",\"url\":\"https://krpraticstorageacc.blob.core.windows.net/azure-openai/cognitive-services-and-machine-learning.md\",\"metadata\":{\"chunking\":\"orignal document size=1188. Scores=5.689296 and None.Org Highlight count=160.Filtering 